In [1]:
import ROOT
from tqdm import tqdm
import pandas as pd
%run ../../utils/helper.ipynb

Welcome to JupyROOT 6.22/06


/opt/root/lib/ROOT/_facade.py:121: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  return _orig_ihook(name, *args, **kwds)
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-pjgd6i9h because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [38]:
channels=["piplus_piplus","piplus_piminus","piminus_piminus","piplus_pi0","pi0_pi0","piminus_pi0"]
versions=["Fall2018_RGA_inbending","Fall2018_RGA_outbending","Spring2019_RGA_inbending","MC_RGA_inbending","MC_RGA_outbending"]
_channels=["$"+c.replace("_","").replace("piplus","\pi^{+}").replace("piminus","\pi^{-}").replace("pi0","\pi^{0}")+"$" for c in channels]
_versions=["Fall 2018 RG-A Inbending", "Fall 2018 RG-A Outbending","Spring 2019 RG-A Inbending", "MC RG-A Inbending","MC RG-A Outbending"]
cuts=["v6","v3","v6","v4","v1","v7"]

In [39]:
def get_MC_cut(channel):
    if(channel=="piplus_piminus"):
        return "truepid_1==211&&truepid_2==-211"
    elif(channel=="piplus_piplus"):
        return "truepid_1==211&&truepid_2==211"
    elif(channel=="piplus_pi0"):
        return "truepid_1==211&&trueparentpid_2==111"
    elif(channel=="piminus_pi0"):
        return "truepid_1==-211&&trueparentpid_2==111"
    elif(channel=="pi0_pi0"):
        return "trueparentpid_1==111&&trueparentpid_2==111"
    elif(channel=="piminus_piminus"):
        return "truepid_1==-211&&truepid_2==-211"

In [40]:
def get_my_cuts(channel,flag,cut_name):
    
    ML_cuts=[]
    for p in pars:
        if("p_" in p):
            ML_cuts.append(f"{p}>0.9")

    Trad_cuts=["isGoodEventWithoutML==1"]

    Base_cuts=[]
    if("pi0" in channel):
        ML_cuts.append("M2>0.106&&M2<0.164")
        Trad_cuts.append("M2>0.106&&M2<0.164")
        Base_cuts.append("M2>0.106&&M2<0.164")
        if("pi0_pi0" in channel):
            ML_cuts.append("M1>0.106&&M1<0.164")
            Trad_cuts.append("M1>0.106&&M1<0.164")
            Base_cuts.append("M1>0.106&&M1<0.164")
    
    _v,_lo,_hi,_ = get_cuts(channel,cut_name)
    for v,lo,hi in zip(_v,_lo,_hi):
        if("p_" in v):
            continue
        ML_cuts.append(f"{v}>{lo}&&{v}<{hi}")
        Trad_cuts.append(f"{v}>{lo}&&{v}<{hi}")
        Base_cuts.append(f"{v}>{lo}&&{v}<{hi}")
        
    if(flag=="true"):
        ML_cuts.append(get_MC_cut(channel))
        Trad_cuts.append(get_MC_cut(channel))
        Base_cuts.append(get_MC_cut(channel))
    elif(flag=="false"):
        ML_cuts.append("!("+get_MC_cut(channel)+")")
        Trad_cuts.append("!("+get_MC_cut(channel)+")")
        Base_cuts.append("!("+get_MC_cut(channel)+")")

    ML_cuts="&&".join(ML_cuts)
    Trad_cuts="&&".join(Trad_cuts)
    Base_cuts="&&".join(Base_cuts)

    return ML_cuts,Trad_cuts,Base_cuts

In [41]:
data = []
for v,_v in zip(versions,_versions):
    for cut,c,_c in tqdm(zip(cuts,channels,_channels)):
        root_file = "../../projects/ana_v2/volatile/data/"+c+f"/{v}_merged.root"
        file = ROOT.TFile(root_file)
        tree = file.Get("dihadron")
        
        if("MC" in v):
            for flag in ["","true","false"]:
                ML_cuts,Trad_cuts,Base_cuts = get_my_cuts(c,flag,cut)
                data.append({"version": _v+" "+flag+" pid", "channel": _c, "entries": tree.GetEntries(Base_cuts), "Egcut_entries": tree.GetEntries(Trad_cuts), "ML_entries": tree.GetEntries(ML_cuts)})
        else:
            ML_cuts,Trad_cuts,Base_cuts = get_my_cuts(c,"",cut)
            data.append({"version": _v, "channel": _c, "entries": tree.GetEntries(Base_cuts), "Egcut_entries": tree.GetEntries(Trad_cuts), "ML_entries": tree.GetEntries(ML_cuts)})

6it [04:46, 47.74s/it]
6it [17:51, 178.57s/it]
6it [09:32, 95.39s/it] 
6it [15:09, 151.53s/it]
6it [11:30, 115.02s/it]


In [42]:
df = pd.DataFrame(data)

In [43]:
# Define alternating colors for table rows
color1 = "blue!10"
color2 = "blue!20"

# Group DataFrame by version
grouped = df.groupby("version")

# Write LaTeX tables to file
with open("dihadron_counts_tables_2.tex", "w") as f:
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage[table]{xcolor}\n")
    f.write("\\usepackage[margin=1in]{geometry}\n")
    f.write("\\begin{document}\n")

    for version, group in grouped:
        # Create list of colors for each row
        colors = [color1, color2] * ((len(group) // 2) + 1)

        # Write LaTeX table code to file
        f.write(f"\\section*{{{version}}}\n")
        f.write("\\begin{center}\n")
        f.write("\\begin{tabular}{ccccc}\n")
        f.write("\\hline\n")
        f.write("Version & Channel & Total Entries & Legacy Entries & ML Entries \\\\\n")
        f.write("\\hline\n")
        for i, row in group.iterrows():
            color = colors[i % len(colors)]
            f.write("\\cellcolor{" + color + "}" + str(row["version"]) + " & ")
            f.write("\\cellcolor{" + color + "}" + str(row["channel"]) + " & ")
            entries_str = "{:.1f}M".format(row["entries"] / 1e6) if row["entries"] >= 1e6 else "{:.1f}K".format(row["entries"] / 1e3)
            f.write("\\cellcolor{" + color + "}" + entries_str + " & ")
            Eg_entries_str = "{:.1f}M".format(row["Egcut_entries"] / 1e6) if row["Egcut_entries"] >= 1e6 else "{:.1f}K".format(row["Egcut_entries"] / 1e3)
            f.write("\\cellcolor{" + color + "}" + (Eg_entries_str if "0" in row["channel"] else "---") + " & ")
            ml_entries_str = "{:.1f}M".format(row["ML_entries"] / 1e6) if row["ML_entries"] >= 1e6 else "{:.1f}K".format(row["ML_entries"] / 1e3)
            f.write("\\cellcolor{" + color + "}" + (ml_entries_str if "0" in row["channel"] else "---") + " \\\\\n")
            f.write("\\hline\n")
        
        f.write("\\end{tabular}\n")
        f.write("\\end{center}\n")

    f.write("\\end{document}\n")

# Convert LaTeX file to PDF using pdflatex
import os
os.system("pdflatex dihadron_counts_tables_2.tex")

0

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./dihadron_counts_tables_2.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/xcolor/xcolor.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/color.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)
(/usr/share/texlive/texmf-dist/tex/latex/colortbl/colortbl.sty
(/usr/share/texlive/texmf-dist/tex/latex/tools/array.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))


# Comparing EventTree
---

In [4]:
import ROOT
import numpy as np

In [9]:
def get_n_dihadrons(runNumber=None,file_name=None):
    if(runNumber!=None):
        file = ROOT.TFile(f"../../projects/ana_v1/volatile/data/piplus_piminus/nSidis_RGA_{runNumber}.root")
    elif(file_name!=None):
        file = ROOT.TFile(file_name)
    tree = file.Get("EventTree")
    N=tree.GetEntries()
    Npiplus=0
    Npiminus=0
    Ndihadrons=0
    for i,iev in enumerate(tree):
        pid=np.array(iev.pid)
        piplus_mask = pid == 211
        piminus_mask = pid == -211
        _nplus = np.count_nonzero(piplus_mask)
        _nminus = np.count_nonzero(piminus_mask)
        Npiplus += _nplus
        Npiminus += _nminus
        Ndihadrons += _nplus*_nminus
    print("Number of Events:",N)
    print("Number of Piplus:",Npiplus)
    print("Number of Piminus:",Npiminus)
    print("Number of Dihadrons:",Ndihadrons)
    print("Dihadrons per event:",np.round(Ndihadrons/N,2))

In [36]:
get_n_dihadrons(5036)
N_5036=43208697

Number of Events: 41747
Number of Piplus: 47065
Number of Piminus: 42633
Number of Dihadrons: 48135
Dihadrons per event: 1.15


In [34]:
print(41747/N_5036)

0.0009661712316851397


In [20]:
get_n_dihadrons(5556)
N_5556=112105269

Number of Events: 238240
Number of Piplus: 249913
Number of Piminus: 251574
Number of Dihadrons: 264286
Dihadrons per event: 1.11


In [35]:
print(238240/N_5556)

0.0021251454291590878


---

In [12]:
get_n_dihadrons(file_name="../../macros/hipo2tree_5354.root")

Number of Events: 76192
Number of Piplus: 85071
Number of Piminus: 77636
Number of Dihadrons: 86768
Dihadrons per event: 1.14


In [13]:
get_n_dihadrons(file_name="../../macros/hipo2tree_5649.root")

Number of Events: 216597
Number of Piplus: 227307
Number of Piminus: 228904
Number of Dihadrons: 240624
Dihadrons per event: 1.11


---

In [31]:
import sys
sys.path.append("/work/clas12/users/gmat/packages/clas12root/rcdb/python/")
import rcdb
# Import necessary modules
import os
import re
from tqdm import tqdm
from rcdb.provider import RCDBProvider
from rcdb.model import ConditionType

db = RCDBProvider("mysql://rcdb@clasdb/rcdb")

In [37]:
def get_total_events_from_dir(directory):
    files = os.listdir(directory)

    run_numbers = []
    run_events = []
    for filename in tqdm(files):
        if "nSidis_00" in filename and filename.endswith(".hipo"):
            run_number = int(filename.split("nSidis_00")[1][:4])
            run_numbers.append(run_number)
            run_events.append(get_events_from_run(run_number))
    print("Nfiles =",len(run_numbers))
    print("Total Events = {:.3e}".format(np.sum(run_events)))
    return run_numbers,run_events

In [38]:
def get_events_from_run(run):
    DB = db.get_run(run)
    return DB.get_condition("event_count").value

In [39]:
f18_inbending_dir = "/cache/clas12/rg-a/production/recon/fall2018/torus-1/pass1/v1/dst/train/nSidis/"
f18_outbending_dir = "/cache/clas12/rg-a/production/recon/fall2018/torus+1/pass1/v1/dst/train/nSidis/"

In [40]:
rn_inb,re_inb=get_total_events_from_dir(f18_inbending_dir)

100%|████████████████████████████████████████| 174/174 [00:03<00:00, 46.93it/s]


Nfiles = 174
Total Events = 1.030e+10


In [41]:
rn_outb,re_outb=get_total_events_from_dir(f18_outbending_dir)

100%|████████████████████████████████████████| 186/186 [00:04<00:00, 45.45it/s]


Nfiles = 186
Total Events = 1.338e+10


In [44]:
def find_closest(list1, list2):
    min_diff = float('inf')
    min_elem1 = None
    min_elem2 = None
    for elem1 in list1:
        for elem2 in list2:
            diff = abs(elem1 - elem2)
            if diff < min_diff:
                min_diff = diff
                min_elem1 = elem1
                min_elem2 = elem2
    return list1.index(min_elem1)

In [47]:
idx=find_closest(re_inb,re_outb)
print(rn_inb[idx],rn_outb[idx])

5354 5649


In [48]:
get_n_dihadrons(5354)

Number of Events: 76187
Number of Piplus: 85066
Number of Piminus: 77631
Number of Dihadrons: 86763
Dihadrons per event: 1.14


In [50]:
print(re_inb[idx])

100001531


In [49]:
get_n_dihadrons(5649)

Number of Events: 216588
Number of Piplus: 227298
Number of Piminus: 228895
Number of Dihadrons: 240615
Dihadrons per event: 1.11


In [51]:
print(re_outb[idx])

100108833


---

In [4]:
import hipopy.hipopy
import numpy as np
import ROOT
from particle import Particle
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.style.use('science')
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [11]:
def get_n_dihadrons_hipopy(file):
    file = hipopy.hipopy.open(file,mode='r')
    file.readBank('REC::Particle')
    file.getNamesAndTypes('REC::Particle')
    
    Npiplus=0
    Npiminus=0
    Ndihadrons=0
    N=0
    for i,event in tqdm(enumerate(file)):
        RECpid = np.array(file.getInts("REC::Particle","pid"))
        if(np.sum(RECpid==11)==0):
            continue
        if(np.sum(RECpid==211)==0):
            continue
        if(np.sum(RECpid==-211)==0):
            continue
        n1=np.sum(RECpid==211)
        n2=np.sum(RECpid==-211)
        Npiplus+=n1
        Npiminus+=n2
        Ndihadrons+=(n1*n2)
        N+=1
    print("Number of Events:",N)
    print("Number of Piplus:",Npiplus)
    print("Number of Piminus:",Npiminus)
    print("Number of Dihadrons:",Ndihadrons)
    print("Dihadrons per event:",np.round(Ndihadrons/N,2))

In [12]:
get_n_dihadrons_hipopy('/cache/clas12/rg-a/production/recon/fall2018/torus-1/pass1/v1/dst/train/nSidis/nSidis_005354.hipo')

693136it [53:41, 215.18it/s]


KeyboardInterrupt: 

Error in callback <bound method StreamCapture.post_execute of <JupyROOT.helpers.utils.StreamCapture object at 0x7f4dc69f0070>> (for post_execute):


KeyboardInterrupt: 

In [ ]:
get_n_dihadrons_hipopy('/cache/clas12/rg-a/production/recon/fall2018/torus+1/pass1/v1/dst/train/nSidis/nSidis_005649.hipo')